In [26]:
!pip install PyMySQL

In [1]:
from flask import Flask,render_template,request  ## flask 라이브러리에서 Flask import
import pymysql
import pandas as pd

In [10]:
app = Flask(__name__)

con = pymysql.connect(host='project-db-campus.smhrd.com', user='sc_24K_bigdata25_p3_4', password='smhrd4',
                      port=3312, db='sc_24K_bigdata25_p3_4', charset='utf8', # 한글처리 (charset = 'utf8')
                      autocommit=True # 결과 DB 반영 (Insert or update)
                     )
cur = con.cursor()

# @app.route('/boot/ex/<int:seqidx>/<user_id>')
# def hello_world(seqidx,user_id):
#     return render_template('eltag.html')

# @app.route('/boot/ex/')
# def hello_world(seqidx,user_id):
#     return render_template('eltag.html')



if __name__ == "__main__":
    app.run(host='localhost', port=8097)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:8097
Press CTRL+C to quit


SyntaxError: invalid syntax (2409517804.py, line 1)

In [ ]:
from flask import Flask, render_template, Response
import cv2
import mediapipe as mp
import pandas as pd
import datetime
import time

app = Flask(__name__)

# MediaPipe 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# 랜드마크 선택
selected_landmarks = [0] + list(range(11, 17)) + list(range(23, 29))

# 컬럼 이름 정의
columns = ['frame'] + [f'landmark{idx}_{axis}' for idx in selected_landmarks for axis in ['x', 'y']] + \
          ['neck_x', 'neck_y', 'hip_x', 'hip_y']

data = []

def generate_frames():
    cap = cv2.VideoCapture(0)  # 카메라 열기
    frame_idx = 0
    start_time = time.time()
    duration = 30  # 30초 동안 실행

    while cap.isOpened() and time.time() - start_time < duration:
        ret, frame = cap.read()
        if not ret:
            break

        frame_idx += 1
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        frame_data = [frame_idx]

        if results.pose_landmarks:
            landmarks = results.pose_landmarks.landmark
            for idx in selected_landmarks:
                landmark = landmarks[idx]
                frame_data.extend([landmark.x, landmark.y])

            # 목과 엉덩이 좌표 계산
            neck_x = (landmarks[11].x + landmarks[12].x) / 2
            neck_y = (landmarks[11].y + landmarks[12].y) / 2
            hip_x = (landmarks[23].x + landmarks[24].x) / 2
            hip_y = (landmarks[23].y + landmarks[24].y) / 2
            frame_data.extend([neck_x, neck_y, hip_x, hip_y])

        else:
            frame_data.extend([None, None] * len(selected_landmarks))
            frame_data.extend([None, None, None, None])

        data.append(frame_data)

        # 시각화
        if results.pose_landmarks:
            for idx in selected_landmarks:
                x, y = int(landmarks[idx].x * frame.shape[1]), int(landmarks[idx].y * frame.shape[0])
                cv2.circle(frame, (x, y), 5, (255, 0, 0), -1)

        _, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

    cap.release()
    pose.close()

    # CSV 저장
    user_id = 'id123'
    suffix = datetime.datetime.now().strftime('%y%m%d_%H%M%S')
    output_csv_path = f'./BPE/training/{user_id}_{suffix}.csv'
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(output_csv_path, index=False)
    print(f"Joint 데이터를 {output_csv_path}에 저장했습니다.")


@app.route('/video_feed')
def video_feed():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/')
def index():
    return render_template('ex-edit.html')

if __name__ == "__main__":
    app.run(host='localhost', port=8097)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://localhost:8097
Press CTRL+C to quit
127.0.0.1 - - [08/Dec/2024 22:33:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2024 22:33:51] "GET /img/아동자세.png HTTP/1.1" 404 -
127.0.0.1 - - [08/Dec/2024 22:33:52] "GET /static/css/index10.css HTTP/1.1" 304 -
127.0.0.1 - - [08/Dec/2024 22:33:58] "GET /video_feed HTTP/1.1" 200 -
127.0.0.1 - - [08/Dec/2024 22:33:58] "GET /favicon.ico HTTP/1.1" 404 -
